In [ ]:
import netket as nk
import numpy as np

In [ ]:
g = nk.graph.Hypercube(length=20, n_dim=1, pbc=True)

In [ ]:
hi = nk.hilbert.Spin(s=1 / 2, N=g.n_nodes)
ha = nk.operator.Ising(hilbert=hi, graph=g, h=1.0)

In [ ]:
ma = nk.models.RBM(alpha=1, dtype=float)

sa = nk.sampler.MetropolisLocal(hi, n_chains=16)

In [ ]:
op = nk.optimizer.Sgd(learning_rate=0.01)

In [ ]:
gs = nk.VMC(ha, op, sa, ma, n_samples=1000)

gs.run(n_iter=300, out=None)